<a href="https://colab.research.google.com/github/williamkbc/sisfinance/blob/master/PosicaoDaycoval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transforma o relatório de posição de cotistas do Daycoval no padrão para importação do arquivo no JCOT


Acessar o portal do Daycoval para baixar o arquivo de "Posição de Cotistas" do dia da transferência
[Portal do Daycoval](https://ecode.daycoval.com.br/?status=2)


*   Baixar o arquivo de "Posicao de Cotistas" no Daycoval Formato TXTUS
*   O arquivo TXTUS utiliza ponto no valor, seguindo o padrão dos arquivos de importação do JCOT
*   O CNPJ mantém o formato



In [273]:
# Conectar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Instalar e configurar PySpark
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import coalesce, lit, to_date
from pyspark.sql.types import DateType



# Criar sessão Spark
spark = SparkSession.builder \
    .appName("LeituraCSVDrive") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Caminho do arquivo CSV da base Posição Cotistas do Daycoval
caminho_csv = '/content/drive/My Drive/PosicaodeCotistasDaycoval/'

# Ler CSV com PySpark SQL
try:
    df_data = spark.read \
        .format("csv") \
        .option("header", "false") \
        .option("inferSchema", "false") \
        .option("delimiter", "\t") \
        .load(caminho_csv)

    print("Leitura realizada com sucesso!")
    df_data.show(5)  # Exibe as primeiras linhas

    df_data.printSchema()

    # Criar view temporária para SQL
    df_data.createOrReplaceTempView("PosicaoCotistas")


    # Exemplo de consulta SQL
    resultado = spark.sql("""
        SELECT *
        FROM PosicaoCotistas
        LIMIT 10
    """)

    print("\nResultado da consulta SQL:")
    resultado.show()

except Exception as e:
    print(f"Erro: {str(e)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Leitura realizada com sucesso!
+---+-----+---+----------------+--------------------+----------+----------+---+---+---+----+--------------------+--------------------+----+-----+------+----------+------+---------+--------+---------+--------------------+----------+----------+----------+----------+----------+----------+----+----+----+---------+---------+------+------+-------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----------+----+----+--------------+----+----+----+--------------+----+
|_c0|  _c1|_c2|             _c3|                 _c4|       _c5|       _c6|_c7|_c8|_c9|_c10|                _c11|                _c12|_c13| _c14|  _c15|      _c16|  _c17|     _c18|    _c19|     _c20|                _c21|      _c22|      _c23|      _c24|      _c25|      _c26|      _c27|_c28|_c29|_c30|     _c31|     _c32|  _c33|  _c34|         _c35|_c36|

## Renomeando o cabeçalho do dataframe

In [274]:
# Usar novo dataframe para renomear cabeçalho conforme CSV do Daycoval
novos_nomes = ["0", "P0045", "C", "NomeCCrt", "NomeLCrt", "DataAtual", "CODataAtual", "CODecNumCota", "CODecValCota", "CODiasCarenciaRend" , "TpIntCrt", "NomeCli", "Endereco", "DigitoAlfa" , "Assessor", "Assessor2", "DataIngresso", "SaldoCotasAtual", "SaldoInveAtual", "Carteira", "Cliente", "CidEstCEP", "DataPos", "DataApl", "ProxAniv", "UltAniv", "ValorAtu", "ValorLiq", "IR", "IOF", "Taxas", "Investimento", "SaldoInv", "SaldoCotas", "Rent", "CotaAtual", "bAplBloq", "bEstqAnt", "bTrataCCDI", "ValorAtuAnt", "ValorLiqAnt", "IRAnt", "IOFAnt", "TaxasAnt", "InvestimentoAnt", "SaldoInvAnt", "SaldoCotasAnt", "CritrApurIR", "CotaMedia", "Rendimento", "BaseIR", "CdIf", "CPF_CNPJ", "Administrador", "CPFCNPJAdm", "TpPlayAdm", "CPFCNPJCliente", "CPFCNPJDistribuidor"]

# Verifica se o número de novos nomes coincide com o número de colunas
if len(novos_nomes) == len(df_data.columns):
    df_posicaocotistas = df_data.toDF(*novos_nomes)
    # Mostra o DataFrame com todas as colunas renomeadas
    df_posicaocotistas.show(5)
else:
    print("Erro: O número de novos nomes não coincide com o número de colunas no DataFrame de dados.")

+---+-----+---+----------------+--------------------+----------+-----------+------------+------------+------------------+--------+--------------------+--------------------+----------+--------+---------+------------+---------------+--------------+--------+---------+--------------------+----------+----------+----------+----------+----------+----------+---+---+-----+------------+---------+----------+------+-------------+--------+--------+----------+-----------+-----------+-----+------+--------+---------------+-----------+-------------+-----------+---------+----------+------+----+--------------+-------------+----------+---------+--------------+-------------------+
|  0|P0045|  C|        NomeCCrt|            NomeLCrt| DataAtual|CODataAtual|CODecNumCota|CODecValCota|CODiasCarenciaRend|TpIntCrt|             NomeCli|            Endereco|DigitoAlfa|Assessor|Assessor2|DataIngresso|SaldoCotasAtual|SaldoInveAtual|Carteira|  Cliente|           CidEstCEP|   DataPos|   DataApl|  ProxAniv|   UltAniv| 

Filtrar as informações para criar a planilha de cadastro de cotistas

In [275]:

df_posicaocotistas.createOrReplaceTempView("cotistas")

df_cotista = spark.sql("""
  SELECT  DISTINCT NomeCli,
                   CPFCNPJCliente,
                   DataIngresso,
                   CASE WHEN LENGTH(CPFCNPJCliente) <= 11 THEN 'PF'
                        WHEN LENGTH(CPFCNPJCliente) >= 12 THEN 'PJ'
                        ELSE 'Outro'
                   END AS TipoCliente
  from cotistas
  GROUP BY NomeCli, CPFCNPJCliente, DataIngresso

      """)

df_cotista.show(100)

+--------------------+--------------+------------+-----------+
|             NomeCli|CPFCNPJCliente|DataIngresso|TipoCliente|
+--------------------+--------------+------------+-----------+
|FACT ENTERPRISE F...|17867471000102|  08/22/2024|         PJ|
|JOAO VINICIUS PRI...|   24848181810|  04/11/2024|         PF|
|   ARMANDO VERISSIMO|   31144513812|  04/15/2024|         PF|
|BRUNO MENDONCA PI...|    8112049602|  04/11/2024|         PF|
|SPARTAN FUNDO DE ...| 9305287000132|  08/22/2024|         PJ|
|FLAVIA PERMAN TEN...|    1992077479|  04/15/2024|         PF|
|  LARISSA BORENSTEIN|   15164684810|  08/22/2024|         PF|
|JOAQUIM PEDRO MON...|    8508146779|  01/22/2025|         PF|
|CELI ELISABETE JU...|   42891280768|  01/22/2025|         PF|
|       ANDRE GHELMAN|    8630796790|  04/16/2024|         PF|
|PEDRO AUGUSTO PRO...|   25022314851|  08/22/2024|         PF|
|         ANDRE BIAGI|    1981661840|  04/15/2024|         PF|
|HELENA JATOBA BRE...|    8393972400|  08/22/2024|     

In [276]:
df_posicaocotistas.createOrReplaceTempView("posicaocotistas")

movimento = spark.sql("""
SELECT
    NomeCCrt AS FUNDO,
    NomeCli AS COTISTA,
    CPFCNPJCliente,
    Investimento,
    DataApl,
    SaldoCotas
    ValorAtu,
    IR,
    IOF,
    CASE
        WHEN LENGTH(CPFCNPJCliente) <= 11 THEN 'PF'
        WHEN LENGTH(CPFCNPJCliente) = 14 THEN 'PJ'
        ELSE 'Outro'
    END AS TipoCliente
FROM posicaocotistas

      """)

movimento.show(50)

+----------------+--------------------+--------------+------------+----------+---------+--------+---+-----------+
|           FUNDO|             COTISTA|CPFCNPJCliente|Investimento|   DataApl| ValorAtu|      IR|IOF|TipoCliente|
+----------------+--------------------+--------------+------------+----------+---------+--------+---+-----------+
|VIA APPIA FIP CB|MAS FUNDO DE INVE...|33632411000159|    300000.0|04/15/2024|    300.0|     0.0|0.0|         PJ|
|VIA APPIA FIP CB|CARLOS EUGENIO DE...|   18477330425|   5000000.0|04/15/2024|   5000.0|     0.0|0.0|         PF|
|VIA APPIA FIP CB|NELSON DA SILVA C...|   13126213898|   2167000.0|04/16/2024|   2167.0|     0.0|0.0|         PF|
|VIA APPIA FIP CB|JULIANA MARIA TAL...|   39038301863|   2167000.0|04/16/2024|   2167.0|     0.0|0.0|         PF|
|VIA APPIA FIP CB|REYNALDO QUARTIM ...|    5098114853|   2167000.0|04/16/2024|   2167.0|     0.0|0.0|         PF|
|VIA APPIA FIP CB|FLAVIO OGNIBENE G...|    5613979880|   15000000.|04/16/2024|  15000.0|

### Valor Total do Fundo

In [291]:
df_posicaocotistas.createOrReplaceTempView("posicaoconsolidada")

consolidada = spark.sql("""
SELECT
  NomeCCrt AS CD_FUNDO,
  CAST(SUM(SaldoInveAtual)AS DECIMAL(12,2)) AS SaldoInveAtual,
  CAST(SUM(ValorAtu) AS DECIMAL(12,2)) AS VL_PATRIMONIO_FECHAMENTO_TOTAL,
  CAST(SUM(ValorLiq) AS DECIMAL(12,2)) AS ValorLiq,
  CAST(SUM(IR) AS DECIMAL(12,2)) AS IR,
  CAST(SUM(IOF) AS DECIMAL(12,2)) AS IOF,
  CAST(SUM(Investimento) AS DECIMAL(12,2)) AS Investimento,
  CAST(SUM(SaldoCotas) AS DECIMAL(18,8)) AS QT_COTAS_FECHAMENTO_TOTAL,
  CotaAtual AS VL_COTA_FECHAMENTO
FROM posicaoconsolidada
GROUP BY NomeCCrt, CotaAtual
      """)

consolidada.show()

+----------------+--------------+------------------------------+-------------+-----------+----+------------+-------------------------+------------------+
|        CD_FUNDO|SaldoInveAtual|VL_PATRIMONIO_FECHAMENTO_TOTAL|     ValorLiq|         IR| IOF|Investimento|QT_COTAS_FECHAMENTO_TOTAL|VL_COTA_FECHAMENTO|
+----------------+--------------+------------------------------+-------------+-----------+----+------------+-------------------------+------------------+
|VIA APPIA FIP CB| 1120700589.15|                 1257248435.59|1232652768.24|24595667.35|0.00|841600589.15|          841594.33332999|     1493.88890325|
+----------------+--------------+------------------------------+-------------+-----------+----+------------+-------------------------+------------------+



Exporta dataframe movimento para o Google Drive

In [ ]:
from datetime import datetime
from google.colab import drive
import os
import shutil
import pandas as pd
# Certifique-se de que openpyxl está instalado para escrever arquivos .xlsx
!pip install openpyxl

agora = datetime.now()

# Formatar a data e hora em uma string adequada para nome de arquivo
formato_data_hora = agora.strftime("_%Y%m%d_%H%M%S")

# Montar o Google Drive (já feito em células anteriores, mas pode ser repetido por segurança)
# drive.mount('/content/drive')

# Definir o caminho de saída para o arquivo XLSX
# O nome do arquivo incluirá a data e hora
caminho_base_saida = '/content/drive/My Drive/SaidaJCOT/'
nome_arquivo_xlsx = f"movimento_cotistas_daycoval{formato_data_hora}.xlsx"
caminho_saida_movimento_xlsx = caminho_base_saida + nome_arquivo_xlsx

# Convert the PySpark DataFrame 'movimento' to a Pandas DataFrame
# Ensure 'movimento' is the correct DataFrame you want to export.
# This step can be memory intensive for large DataFrames.
try:
    # Coalesce to a single partition before converting to reduce memory pressure
    # on a single worker/driver if the PySpark DataFrame is partitioned.
    # However, be aware that coalescing can be slow for large DataFrames.
    # If the DataFrame is already small, you might skip coalesce(1).
    pandas_df_movimento = movimento.toPandas()
    print("DataFrame PySpark convertido para Pandas com sucesso.")

    # Save the Pandas DataFrame to an XLSX file
    pandas_df_movimento.to_excel(caminho_saida_movimento_xlsx, index=False) # index=False avoids writing the DataFrame index as a column
    print(f"\nDataFrame 'movimento' salvo com sucesso em: {caminho_saida_movimento_xlsx}")

except Exception as e:
    print(f"\nErro ao converter ou salvar o DataFrame: {str(e)}")

# Note: Since we are writing directly to an XLSX file using Pandas,
# there is no temporary directory created by PySpark's write method,
# so no need to remove a directory afterwards.

DataFrame PySpark convertido para Pandas com sucesso.

DataFrame 'movimento' salvo com sucesso em: /content/drive/My Drive/SaidaJCOT/movimento_cotistas_daycoval_20250520_174144.xlsx


In [ ]:
from datetime import datetime
from google.colab import drive
import os
import shutil
import pandas as pd
# Certifique-se de que openpyxl está instalado para escrever arquivos .xlsx
!pip install openpyxl

agora = datetime.now()

# Formatar a data e hora em uma string adequada para nome de arquivo
formato_data_hora = agora.strftime("_%Y%m%d_%H%M%S")

# Montar o Google Drive (já feito em células anteriores, mas pode ser repetido por segurança)
# drive.mount('/content/drive')

# Definir o caminho de saída para o arquivo XLSX
# O nome do arquivo incluirá a data e hora
caminho_base_saida = '/content/drive/My Drive/SaidaJCOT/'
nome_arquivo_xlsx = f"cotista_daycoval{formato_data_hora}.xlsx"
caminho_saida_cotista_xlsx = caminho_base_saida + nome_arquivo_xlsx

# Convert the PySpark DataFrame 'movimento' to a Pandas DataFrame
# Ensure 'movimento' is the correct DataFrame you want to export.
# This step can be memory intensive for large DataFrames.
try:
    # Coalesce to a single partition before converting to reduce memory pressure
    # on a single worker/driver if the PySpark DataFrame is partitioned.
    # However, be aware that coalescing can be slow for large DataFrames.
    # If the DataFrame is already small, you might skip coalesce(1).
    pandas_df_cotista = df_cotista.toPandas()
    print("DataFrame PySpark convertido para Pandas com sucesso.")

    # Save the Pandas DataFrame to an XLSX file
    pandas_df_cotista.to_excel(caminho_saida_cotista_xlsx, index=False) # index=False avoids writing the DataFrame index as a column
    print(f"\nDataFrame 'cotista' salvo com sucesso em: {caminho_saida_cotista_xlsx}")

except Exception as e:
    print(f"\nErro ao converter ou salvar o DataFrame: {str(e)}")

# Note: Since we are writing directly to an XLSX file using Pandas,
# there is no temporary directory created by PySpark's write method,
# so no need to remove a directory afterwards.

DataFrame PySpark convertido para Pandas com sucesso.

DataFrame 'cotista' salvo com sucesso em: /content/drive/My Drive/SaidaJCOT/cotista_daycoval_20250520_174152.xlsx


### Dados consolidados do fundo

In [288]:
from datetime import datetime
from google.colab import drive
import os
import shutil
import pandas as pd
# Certifique-se de que openpyxl está instalado para escrever arquivos .xlsx
!pip install openpyxl

agora = datetime.now()

# Formatar a data e hora em uma string adequada para nome de arquivo
formato_data_hora = agora.strftime("_%Y%m%d_%H%M%S")


# Definir o caminho de saída para o arquivo XLSX
# O nome do arquivo incluirá a data e hora
caminho_base_saida = '/content/drive/My Drive/SaidaJCOT/'
nome_arquivo_xlsx = f"fundo_consolidado{formato_data_hora}.xlsx"
caminho_saida_consolidada_xlsx = caminho_base_saida + nome_arquivo_xlsx

# Convert the PySpark DataFrame 'consolidada' to a Pandas DataFrame
# Ensure 'consolidada' is the correct DataFrame you want to export.
# This step can be memory intensive for large DataFrames.
try:
    # Coalesce to a single partition before converting to reduce memory pressure
    pandas_consolidada = consolidada.toPandas()
    print("DataFrame PySpark convertido para Pandas com sucesso.")

    # Save the Pandas DataFrame to an XLSX file
    pandas_consolidada.to_excel(caminho_saida_consolidada_xlsx, index=False) # index=False avoids writing the DataFrame index as a column
    print(f"\nDataFrame 'cotista' salvo com sucesso em: {caminho_saida_consolidada_xlsx}")

except Exception as e:
    print(f"\nErro ao converter ou salvar o DataFrame: {str(e)}")

# Note: Since we are writing directly to an XLSX file using Pandas,
# there is no temporary directory created by PySpark's write method,
# so no need to remove a directory afterwards.

DataFrame PySpark convertido para Pandas com sucesso.

DataFrame 'cotista' salvo com sucesso em: /content/drive/My Drive/SaidaJCOT/fundo_consolidado_20250520_184417.xlsx
